# SEMA-1D 

SEMA-1D is a fine-tuned ESM-1v model aimed to predict epitope resiudes based on antigen protein sequence

The MIT License (MIT)
Copyright (c) 2016 AYLIEN
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Requirements

In [1]:
import os
# set cuda params
# 'TORCH_HOME'directory will be used to save origenal esm-1v weights
os.environ['TORCH_HOME'] = "../torch_hub"
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

In [2]:
import scipy
import sklearn
import esm

import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torch import nn
import math

import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments, EvalPrediction

from esm.pretrained import load_model_and_alphabet_hub

from sklearn.metrics import r2_score, mean_squared_error

2024-01-30 08:19:33.103874: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-30 08:19:33.486693: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 08:19:35.491819: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-01-30 08:19:35.492072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

## Model and metrics

In [3]:
class PDB_Dataset(Dataset):
    """
    A class to represent a sutable data set for model. 
    
    convert original pandas data frame to model set,
    where 'token ids' is ESM-1v embedings corresponed to protein sequence (max length 1022 AA)
    and 'lables' is a contact number values
    Attributes:
        df (pandas.DataFrame): dataframe with two columns: 
                0 -- preotein sequence in string ('GLVM') or list (['G', 'L', 'V', 'M']) format
                1 -- contcat number values in list [0, 0.123, 0.23, -100, 1.34] format
        esm1v_batch_converter (function):
                    ESM function callable to convert an unprocessed (labels + strings) batch to a
                    processed (labels + tensor) batch.
        label_type (str):
                type of model: regression or binary

    """
    def __init__(self, df, label_type ='regression'):
        """
        Construct all the necessary attributes to the PDB_Database object.
        
        Parameters:
            df (pandas.DataFrame): dataframe with two columns: 
                0 -- protein sequence in string ('GLVM') or list (['G', 'L', 'V', 'M']) format
                1 -- contcat number values in list [0, 0.123, 0.23, -100, 1.34] format
            label_type (str):
                type of model: regression or binary
        """
        self.df = df
        _, esm1v_alphabet = esm.pretrained.esm1v_t33_650M_UR90S_1()
        self.esm1v_batch_converter = esm1v_alphabet.get_batch_converter()
        self.label_type = label_type

    def __getitem__(self, idx):
        item = {}
        _, _, esm1b_batch_tokens = self.esm1v_batch_converter([('' , ''.join(self.df.iloc[idx,0])[:1022])])
        item['token_ids'] = esm1b_batch_tokens
        item['labels'] = torch.unsqueeze(torch.FloatTensor(self.df.iloc[idx, 1][:1022]),0).to(torch.float64)

        return item

    def __len__(self):
        return len(self.df)

In [4]:
class ESM1vForTokenClassification(nn.Module):

    def __init__(self, num_labels = 2, pretrained_no = 1):
        super().__init__()
        self.num_labels = num_labels
        self.model_name = esm.pretrained.esm2_t33_650M_UR50D()  
        
        self.esm1v, self.esm1v_alphabet = esm.pretrained.esm2_t33_650M_UR50D()#load_model_and_alphabet_hub(self.model_name)        
        self.classifier = nn.Linear(1280, self.num_labels)

    def forward(self, token_ids, labels = None):
                
        outputs = self.esm1v.forward(token_ids, repr_layers=[33])['representations'][33]
        outputs = outputs[:,1:-1,:]
        logits = self.classifier(outputs)

        return SequenceClassifierOutput(logits=logits)

In [5]:
def compute_metrics_regr(p: EvalPrediction):
    
    preds = p.predictions[:,:,1]

    batch_size, seq_len = preds.shape    
    out_labels, out_preds = [], []

    for i in range(batch_size):
        for j in range(seq_len):
            if p.label_ids[i, j] > -1:
                out_labels.append(p.label_ids[i][j])
                out_preds.append(preds[i][j])
                
    out_labels_regr = out_labels#[math.log(t+1) for t in out_labels]

    
    return {
        "pearson_r": scipy.stats.pearsonr(out_labels_regr, out_preds)[0],
        "mse": mean_squared_error(out_labels_regr, out_preds),
        "r2_score": r2_score(out_labels_regr, out_preds)
    }


In [6]:
## you can train one model or ensemple of 5 models
def model_init_1():
    return ESM1vForTokenClassification(pretrained_no = 1).cuda()
def model_init_2():
    return ESM1vForTokenClassification(pretrained_no = 2).cuda()
def model_init_3():
    return ESM1vForTokenClassification(pretrained_no = 3).cuda()
def model_init_4():
    return ESM1vForTokenClassification(pretrained_no = 4).cuda()
def model_init_5():
    return ESM1vForTokenClassification(pretrained_no = 5).cuda()

In [7]:
class MaskedMSELoss(torch.nn.Module):
    def __init__(self):
        super(MaskedMSELoss, self).__init__()

    def forward(self, inputs, target, mask):    
        diff2 = (torch.flatten(inputs[:,:,1]) - torch.flatten(target)) ** 2.0 * torch.flatten(mask)
        result = torch.sum(diff2) / torch.sum(mask)
        if torch.sum(mask)==0:
            return torch.sum(diff2)
        else:
            #print('loss:', result)
            return result
        

In [8]:
class MaskedRegressTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        labels = labels.squeeze().detach().cpu().numpy().tolist()
        labels = [math.log(t+1) if t!=-100 else -100 for t in labels]
        labels = torch.unsqueeze(torch.FloatTensor(labels), 0).cuda()
        masks = ~torch.eq(labels, -100).cuda()


        
        #masks = inputs.pop("masks")
        outputs = model(**inputs)
        logits = outputs.logits

        loss_fn = MaskedMSELoss()
        
        loss = loss_fn(logits, labels, masks)
        
        return (loss, outputs) if return_outputs else loss

In [9]:
def collator_fn(x):
    if len(x)==1:
        return x[0]
    print('x:', x)
    return x

## Data

In [10]:
train_set = pd.read_csv('../data/sema_2.0/train_set.csv')
train_set = train_set.groupby('pdb_id_chain').agg({'resi_pos': list,
                                 'resi_aa': list,
                                 'contact_number': list}).reset_index()
## the first run will take about 5-10 minutes, because esm weights should be downloaded
train_ds = PDB_Dataset(train_set[['resi_aa', 'contact_number']], 
                      label_type ='regression')

test_set = pd.read_csv('../data/sema_2.0/test_set.csv')
test_set = test_set.groupby('pdb_id_chain').agg({'resi_pos': list,
                                 'resi_aa': list,
                                 'contact_number': list}).reset_index()
test_ds = PDB_Dataset(test_set[['resi_aa', 'contact_number']],
                      label_type ='regression')

/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


## Model training and test

In [11]:
training_args = TrainingArguments(
    output_dir='./results_fold' ,          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    learning_rate=1e-05,             # learning rate
    weight_decay=0.0,                # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    save_strategy = "no",
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after each epoch
    gradient_accumulation_steps=1,  # total number of steps before back propagation
    fp16=False,                       # Use mixed precision
    run_name="PDB_binary",      # experiment name
    seed=42,                         # Seed for experiment reproducibility
    load_best_model_at_end=False,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
    #remove_unused_columns=False

)

In [12]:
#create direactory to weights storage
if not os.path.exists("../models/"):
    os.makedirs("../models/")

### one model

In [13]:
trainer = MaskedRegressTrainer(
    model=model_init_1(),                   # the instantiated 🤗 Transformers model to be trained
    args=training_args,                     # training arguments, defined above
    train_dataset=train_ds,                 # training dataset
    eval_dataset=test_ds,                   # evaluation dataset
    data_collator = collator_fn,
    compute_metrics = compute_metrics_regr,    # evaluation metrics
)

trainer.train()

#save weights
torch.save(trainer.model.state_dict(), "../models/sema_1d_ESM2_0.pth")

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Pearson R,Mse,R2 Score
1,0.218500,0.171049,0.233485,0.195962,-0.766367
2,0.171600,0.167218,0.241095,0.189915,-0.711853


### ensemble of 5 models

In [14]:
for idx, model_init in enumerate([model_init_1, model_init_2, model_init_3, model_init_4, model_init_5]):
    
    trainer = MaskedRegressTrainer(
        model=model_init(),                   # the instantiated 🤗 Transformers model to be trained
        args=training_args,                   # training arguments, defined above
        train_dataset=train_ds,               # training dataset
        eval_dataset=test_ds,                 # evaluation dataset
        data_collator = collator_fn,
        compute_metrics = compute_metrics_regr    # evaluation metrics
    )

    trainer.train()
    
    #save weights
    torch.save(trainer.model.state_dict(), f"../models/sema_1d_ESM2_{str(idx)}.pth")
    training_args.seed = training_args.seed+1

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Pearson R,Mse,R2 Score
1,0.216000,0.159625,0.241303,0.185884,-0.675527
2,0.173300,0.178247,0.247572,0.198102,-0.785656


Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Pearson R,Mse,R2 Score
1,0.233900,0.172618,0.229943,0.194388,-0.752171
2,0.166500,0.175772,0.240026,0.191759,-0.728481


Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Pearson R,Mse,R2 Score
1,0.229100,0.164949,0.242964,0.187434,-0.689496
2,0.175100,0.165896,0.246877,0.186859,-0.684308


Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Pearson R,Mse,R2 Score
1,0.230200,0.168872,0.242282,0.188585,-0.699865
2,0.169200,0.171217,0.246204,0.190292,-0.715258


Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Pearson R,Mse,R2 Score
1,0.236000,0.161467,0.243766,0.183937,-0.657971
2,0.173800,0.168397,0.255022,0.188486,-0.698972


In [15]:
train_ds[0].pop('labels')

tensor([[-100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000,    0.0000, -100.0000, -100.0000, -100.0000,    0.0000,
            0.0000, -100.0000, -100.0000,    0.0000,    0.0000, -100.0000,
         -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,    0.0000,
            0.0000,    0.0000, -100.0000,    0.0000,    0.0000, -100.0000,
         -100.0000,    0.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000, -100.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.6931,    0.6931,    1.0986,    1.9459,    1.7918,
            1.6094,    2.0794,    2.7081,    2.3026,    1.9459,    2.4849,
            3.0910,    2.

In [16]:
len(train_ds)

783

In [17]:
t = train_ds[0]

In [18]:
t.pop('labels')

tensor([[-100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000,    0.0000, -100.0000, -100.0000, -100.0000,    0.0000,
            0.0000, -100.0000, -100.0000,    0.0000,    0.0000, -100.0000,
         -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,    0.0000,
            0.0000,    0.0000, -100.0000,    0.0000,    0.0000, -100.0000,
         -100.0000,    0.0000, -100.0000, -100.0000, -100.0000, -100.0000,
         -100.0000, -100.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.6931,    0.6931,    1.0986,    1.9459,    1.7918,
            1.6094,    2.0794,    2.7081,    2.3026,    1.9459,    2.4849,
            3.0910,    2.

In [19]:
train_ds[0]

{'token_ids': tensor([[ 0, 14, 12,  7, 16, 17,  4, 16,  6, 16, 20,  7, 21, 16,  5, 12,  8, 14,
          10, 11,  4, 17,  5, 22,  7, 15,  7,  7,  9,  9, 15,  5, 18,  8, 14,  9,
           7, 12, 14, 20, 18,  8,  5,  4,  8,  9,  6,  5, 11, 14, 16, 13,  4, 17,
          11, 20,  4, 17, 11,  7,  6,  6, 21, 16,  5,  5, 20, 16, 20,  4, 15,  9,
          11, 12, 17,  9,  9,  5,  5,  9, 22, 13, 10,  4, 21, 14,  7, 21,  5,  6,
          14, 12,  5, 14,  6, 16, 20, 10,  9, 14, 10,  6,  8, 13, 12,  5,  6, 11,
          11,  8, 11,  4, 16,  9, 16, 12,  6, 22, 20, 11, 21, 17, 14, 14, 12, 14,
           7,  6,  9, 12, 19, 15, 10, 22, 12, 12,  4,  6,  4, 17, 15, 12,  7, 10,
          20, 19,  8, 14, 11,  8, 12,  4,  2]]),
 'labels': tensor([[-100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
          -100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
          -100.0000, -100.0000, -100.0000, -100.0000, -100.0000, -100.0000,
          -100.0000, -100.0000, -100.0000, -